In [15]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from sklearn import cross_validation

First we use Optical Character Recognition (OCR) method to recognize hand-written digits.

In [34]:
img = cv2.imread('digits.png')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

# Now we split the image to 5000 cells, each 20x20 size
cells = [np.hsplit(row,100) for row in np.vsplit(gray,50)]

# Make it into a Numpy array. It size will be (50,100,20,20)
x = np.array(cells)

# Now we prepare train_data and test_data.
train = x[:,:50].reshape(-1,400).astype(np.float32) # Size = (2500,400)
test = x[:,50:100].reshape(-1,400).astype(np.float32) # Size = (2500,400)

# Create labels for train and test data
k = np.arange(10)
train_labels = np.repeat(k,250)[:,np.newaxis]
test_labels = train_labels.copy()

# Initiate kNN, train the data, then test it with test data for k=5
knn = cv2.KNearest()
knn.train(train,train_labels)
ret,result,neighbours,dist = knn.find_nearest(test,k=5) 

# Now we check the accuracy of classification
# For that, compare the result with test_labels and check which are wrong
matches = result==test_labels
correct = np.count_nonzero(matches)
accuracy = correct*100.0/result.size
print accuracy

91.76


In [24]:
train

array([[ 19.,   2.,   8., ...,   8.,   0.,   8.],
       [  8.,   5.,  12., ...,   8.,   4.,  10.],
       [  3.,   4.,  11., ...,   7.,   3.,   9.],
       ..., 
       [  9.,   5.,  11., ...,   7.,   0.,   7.],
       [ 22.,   4.,   9., ...,  11.,   3.,   9.],
       [ 16.,   5.,  10., ...,   8.,   4.,   8.]], dtype=float32)

In [25]:
test

array([[ 22.,   6.,   9., ...,  11.,   0.,   8.],
       [  9.,   2.,   9., ...,   6.,   0.,   8.],
       [  3.,   5.,  10., ...,  10.,   6.,   6.],
       ..., 
       [ 19.,   6.,   9., ...,  12.,   2.,   4.],
       [ 18.,   2.,   3., ...,   9.,   5.,   8.],
       [  0.,   4.,   9., ...,   7.,   2.,   8.]], dtype=float32)

In [36]:
data = np.vstack([train,test])
k = np.arange(10)
label = np.vstack([train_labels,test_labels])
sampNum= len(data) 

kf = cross_validation.KFold(len(data), n_folds=10)
iFold = 0
for train_index, val_index in kf:
    iFold = iFold+1
    X_train, X_val, y_train, y_val = data[train_index], data[val_index], label[train_index], label[val_index]
    knn.train(X_train,y_train)
    ret,result,neighbours,dist = knn.find_nearest(X_val,k=5) 
    matches = result==y_val
    correct = np.count_nonzero(matches)
    accuracy = correct*100.0/result.size
    print accuracy

98.2
85.6
85.2
91.6
85.0
98.0
86.2
82.2
94.2
84.8


In [37]:
np.mean([98.2,85.6,85.2,91.6,85.0,98.0,86.2,82.2,94.2,84.8])

89.099999999999994

In [11]:
# save the data
np.savez('knn_data.npz',train=train, train_labels=train_labels)

# Now load the data
with np.load('knn_data.npz') as data:
    print data.files
    train = data['train']
    train_labels = data['train_labels']

['train_labels', 'train']


Using the letter-recognition data in OpenCv file, we can train KNN model to recognize hand-written letters.

In [13]:
# Load the data, converters convert the letter to a number
data= np.loadtxt('letter-recognition.data', dtype= 'float32', delimiter = ',',
                    converters= {0: lambda ch: ord(ch)-ord('A')})

# split the data to two, 10000 each for train and test
train, test = np.vsplit(data,2)

# split trainData and testData to features and responses
responses, trainData = np.hsplit(train,[1])
labels, testData = np.hsplit(test,[1])

# Initiate the kNN, classify, measure accuracy.
knn = cv2.KNearest()
knn.train(trainData, responses)
ret, result, neighbours, dist = knn.find_nearest(testData, k=5)

correct = np.count_nonzero(result == labels)
accuracy = correct*100.0/10000
print accuracy

93.22
